In [1]:
import pandas as pd

In [2]:
df_cat = pd.read_excel("SPCategories.xlsx")

In [3]:
df_cat  = df_cat.drop(['◢','System.Collections.Generic.List<string>',"spClass.ListCategory"],axis=1)
df_cat  = df_cat.rename(columns={"spClass.ListCategory": "cat_num", "Count = 52": "cat_name"})

df_cat["cat_name"] = df_cat["cat_name"].str.strip('"')
df_cat = df_cat.iloc[:-1]

df_cat.head(7) 

,cat_name
0,
1,Article
2,Audited Financial Statements
3,AUM
4,"Capital: committed, invested"
5,CCO Notes
6,Compliance - Other


In [4]:
df_cat = df_cat.drop(index=0).reset_index(drop=True)

In [5]:
len(df_cat)

51

In [6]:
df_names = pd.read_csv('data.csv', usecols=[0])
df_names.head(7)

,Name
0,PMC Q3 2024.pptx
1,Quarterly Manager Blurb - 2024.xlsx
2,GTF Exposure Snapshot - Sep 2024.xlsx
3,Janus Henderson Biotechnology Innovation Fund ...
4,2024-11-07-Documents By Last Modified Date.xlsx
5,Verition Commentary & Risk Report_September 20...
6,Saba Capital Carry Neutral Tail Fund - Risk Re...


In [7]:
df_names = df_names.fillna("")
#df_names = df_names[df_names["Name"].apply(lambda x: len(x) >= 6)]
df_names

,Name
0,PMC Q3 2024.pptx
1,Quarterly Manager Blurb - 2024.xlsx
2,GTF Exposure Snapshot - Sep 2024.xlsx
3,Janus Henderson Biotechnology Innovation Fund ...
4,2024-11-07-Documents By Last Modified Date.xlsx
...,...
588217,sharedStrings.xml
588218,styles.xml
588219,vbaProject.bin
588220,.rels


In [8]:
import pandas as pd
import numpy as np

# Assuming df_names is a single column with values [1, 2, 3]
# and df_cat is a single column with values ['a', 'b', 'c']

# Create an empty DataFrame with 'df_cat' values as column names
df_result = pd.DataFrame(np.nan, index=df_names.index, columns=df_cat.iloc[:, 0].values)

# Add df_names as the first column
df_result.insert(0, df_names.columns[0], df_names)

# View the result
#df_result = df_result.head(50000)

In [9]:
import string
import re

# List of stop words to remove
stop_words = ["or", "and", "the", "of", "to", "in", "for", "on", "at", "a", "is", "it", "by", "an"]

# Convert column names to lowercase and remove stop words
stop_words_pattern = r'\b(?:' + '|'.join(map(re.escape, stop_words)) + r')\b'

df_result.columns = (
    df_result.columns.str.lower()
    .str.replace(f"[{string.punctuation}]", "", regex=True)
    .str.replace(stop_words_pattern, '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)  # Replace any spaces with underscores
    .str.strip('_')  # Remove leading/trailing underscores
)

df_result

,name,article,audited financial statements,aum,capital committed invested,cco notes,compliance other,compliance business continuity manual,conference call,conference meeting notes,...,reference notes,risk management policy,risk report,service provider information,side letter,tax information,term sheet,unaudited financial statements,valuation policy,wire instructions
0,PMC Q3 2024.pptx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Quarterly Manager Blurb - 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GTF Exposure Snapshot - Sep 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Janus Henderson Biotechnology Innovation Fund ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-07-Documents By Last Modified Date.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588217,sharedStrings.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588218,styles.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588219,vbaProject.bin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588220,.rels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import re
from tqdm import tqdm  # Import tqdm for progress bar
from transformers import BertTokenizer

# Load the tokenizer from Hugging Face (BERT tokenizer in this example)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ensure columns and 'name' values are lowercase
df_result.columns = df_result.columns.str.lower()
df_result['name'] = df_result['name'].str.lower()

# Tokenizer function that splits text into words by any non-alphabetical characters
def tokenize(text):
    # Split by any non-alphabetical character, such as punctuation, spaces, etc.
    words = re.findall(r'\b\w+\b', text)  # \b matches word boundaries (alphabetic characters)
    return words

# Helper function to match base words (using word matching)
def match_base_word(name_tokens, column_tokens):
    for name_token in name_tokens:
        # Check if any column token matches any name token or vice versa
        for column_token in column_tokens:
            if name_token == column_token:
                return True
    return False

# Use tqdm to create a progress bar for iterating over rows
for idx, row in tqdm(df_result.iterrows(), total=df_result.shape[0], desc="Processing rows", unit="row", ncols=100):
    name_tokens = tokenize(row['name'])  # Tokenize the file name using custom tokenizer
    
    # Iterate over all columns except 'name'
    for column in df_result.columns:
        if column != 'name':
            column_tokens = tokenize(column)  # Tokenize the column name using custom tokenizer
            
            # Check for base word matches
            if match_base_word(name_tokens, column_tokens):
                # If there's a match, count the overlap (using words)
                overlap = sum(1 for token in column_tokens if token in name_tokens)
                if overlap > 0:
                    df_result.at[idx, column] = 1  # Add 1 if there is a match

# View the result
print(df_result)

Processing rows: 100%|███████████████████████████████████| 588222/588222 [01:42<00:00, 5722.50row/s]


                                                     name  article  \
0                                        pmc q3 2024.pptx      NaN   
1                     quarterly manager blurb - 2024.xlsx      NaN   
2                   gtf exposure snapshot - sep 2024.xlsx      NaN   
3       janus henderson biotechnology innovation fund ...      NaN   
4         2024-11-07-documents by last modified date.xlsx      NaN   
...                                                   ...      ...   
588217                                  sharedstrings.xml      NaN   
588218                                         styles.xml      NaN   
588219                                     vbaproject.bin      NaN   
588220                                              .rels      NaN   
588221                                       workbook.xml      NaN   

        audited financial statements  aum  capital committed invested  \
0                                NaN  NaN                         NaN   
1            

In [ ]:
# Exclude the 'name' column and check for rows with all NaN values in other columns
nan_rows = df_result.drop(columns=['name']).isna().all(axis=1)

# Count the number of rows that are fully NaN except for the 'name' column
num_nan_rows = nan_rows.sum()

# Print the result
print(f"Number of rows with all NaN values except for 'name': {num_nan_rows}")

In [ ]:
df_result.columns

In [ ]:
df_result = df_result.fillna(0)

In [ ]:
df_result

In [ ]:
"""
# Create the 'Unknown' column and initialize it with 0
df_result['Unknown'] = 0

# Iterate over the rows and check if the sum of the numerical values is zero
for idx, row in df_result.iterrows():
    if row.drop('name').sum() == 0:  # Exclude the 'name' column, sum the rest
        df_result.at[idx, 'Unknown'] = 1  # Set 'Unknown' to 1 if sum is zero
"""

In [ ]:
df_result

In [ ]:
# Create an empty list to store the rows
result_list = []

# Iterate through each row of df_result
for idx, row in df_result.iterrows():
    # Get the name value (row['name'])
    name = row['name']
    
    # Get the values in the columns to the right of 'name'
    values = row.iloc[1:]  # Skipping the first column ('name')

    # Find the highest value in those columns
    highest_value = values.max()

    # Store the result in the list
    result_list.append({'name': name, 'highest_value': highest_value})

# Create the final DataFrame from the list of results
highest_value_df = pd.DataFrame(result_list)

In [ ]:
# Number of word matches

#        Name              Classification
# "fund due diligence" = due || diligence ==> 1
# "fund due diligence" = due && diligence ==> 2

highest_value_df.highest_value.value_counts()

In [ ]:
# Total Matches
highest_value_df.highest_value.value_counts().sum()

In [ ]:
total_files = len(highest_value_df)

# Number of file names with no matches (highest_value is NaN or 0)
no_matches = len(highest_value_df[highest_value_df['highest_value'].isna() | (highest_value_df['highest_value'] == 0)])

# Calculate the percentage of file names with no matches
no_matches_percentage = (no_matches / total_files) * 100

# Output the result
print(f"Percentage of file names with no matches: {no_matches_percentage:.2f}%")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_result is your DataFrame

# Drop the 'name' column and sum the values in each of the other columns
column_sums = df_result.drop('name', axis=1).sum()

# Sort the column sums in descending order
column_sums_sorted = column_sums.sort_values(ascending=False)

# Plot the bar chart
plt.figure(figsize=(10, 6))
column_sums_sorted.plot(kind='bar')
plt.title('Sum of Values in Each Column')
plt.xlabel('Column Names')
plt.ylabel('Sum of Values')

# Rotate the x-axis labels to 90 degrees for better readability
plt.xticks(rotation=90)

# Adjust layout to ensure labels fit properly
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
df_result.to_csv('file_class_full.csv', index=False)
df_result

In [ ]:
import pandas as pd
import re
from openpyxl import load_workbook

# List of stop words to exclude
stop_words = {'the', 'and', 'for', 'on', 'with', 'in', 'a', 'to', 'at', 'by', 'an', 'as', 'it', 'of', 'is', 'from', 'this', 'that', 'which', 'be', '-'}

# Regex pattern to match common file extensions
file_extension_pattern = r'\.(pptx|xlsx|xls|docx|doc|pdf|txt|csv|png|jpg|jpeg|gif|bmp|tiff|xml|zip|rar|7z|tar|gz|log|mp4|mp3|avi|mkv|wav|mov|json|html|htm|css|js|exe|dll|bin|py|java|c|cpp|class|sh|bat|sql|db|bak|log|tmp)$'

# Function to extract words from file names
def extract_words_from_filename(file_name):
    # Remove file extensions
    file_name = re.sub(file_extension_pattern, '', file_name, flags=re.IGNORECASE)
    # Extract words (keep hyphens and underscores as part of the word, treat them as single tokens)
    words = re.findall(r'\b[a-zA-Z\-_\']+\b', file_name.lower())  # Include hyphen and underscore
    # Filter out stop words
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

# Ensure the 'df_result' DataFrame exists and contains data
if 'df_result' in globals() and not df_result.empty:
    # Make a copy of df_result to ensure the original is not overwritten
    df_copy = df_result.copy()

    # Step 1: Identify rows where all values to the right of the 'name' column are zeros
    condition = df_copy.iloc[:, 1:].eq(0).all(axis=1)
    
    # Step 2: Extract words from file names where the condition is True
    df_copy['words'] = df_copy[condition]['name'].apply(extract_words_from_filename)

    # Step 3: Check if words were extracted and filter empty lists
    df_copy = df_copy[df_copy['words'].notnull() & (df_copy['words'].str.len() > 0)]

    # Step 4: Flatten the word lists into a single series
    all_words = df_copy.explode('words')

    # Step 5: Count word frequencies and keep the first occurrence (as example)
    word_counts = all_words.groupby('words').agg(
        count=('words', 'size'),
        example=('name', 'first')  # Take the first filename as the example
    ).reset_index()

    # Step 6: Add an empty 'Classification' column
    word_counts['Classification'] = ""  # Empty classification column

    # Step 7: Export the top 15 words for lengths 3 to 12 into separate sheets
    output_path = 'top_words_unclassified_words.xlsx'
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for length in range(3, 13):  # Word lengths from 3 to 12
            # Filter words with the specific length
            top_words = word_counts[word_counts['words'].str.len() == length].sort_values(by='count', ascending=False).head(15)
            
            # If there are no top words for this length, skip
            if top_words.empty:
                continue
            
            sheet_name = f'{length}_Char_Words'
            
            # Write to Excel in a separate sheet for each length
            top_words.to_excel(writer, sheet_name=sheet_name, index=False)
        
            # Adjust column widths for better readability
            workbook = writer.book
            sheet = workbook[sheet_name]
            sheet.column_dimensions['A'].width = 30  # 'word' column width
            sheet.column_dimensions['B'].width = 10  # 'count' column width
            sheet.column_dimensions['C'].width = 120  # 'example' column width
            sheet.column_dimensions['D'].width = 20  # 'Classification' column width

    print(f"Top words (length 3-12) with examples and classification exported to {output_path}")
else:
    print("No data to process in 'df_result' DataFrame.")


In [ ]:
row_sums = df_result.iloc[:, 1:].sum(axis=1)

# Filter rows where the row sum is not zero
result_df = df_result[row_sums != 0]

# Export the resulting DataFrame to a CSV file
result_df.to_csv('result_df.csv', index=False)


In [ ]:
row_sums = df_result.iloc[:, 1:].sum(axis=1)

# Filter rows where the row sum is zero (excluding the 'name' column)
filtered_df = df_result[row_sums == 0]

# Randomly select 100 rows from the filtered DataFrame
random_sample = filtered_df.sample(n=140, random_state=42)  # random_state for reproducibility

# Only select the 'name' column and filter for rows that contain a period ('.')
filtered_names = random_sample['name'][random_sample['name'].str.contains(r'\.', na=False)]

# Print the filtered names column
print(filtered_names)

# Optionally, export the filtered names column to a CSV file
filtered_names.to_csv('random_files_unclassified.csv', index=False)